In [36]:
import spacy
from spacy.matcher import Matcher
from zipfile import ZipFile
from pathlib import Path
from tqdm import autonotebook as tqdm
import os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from spacy.tokens import Doc
import re
from collections import Counter
from collections.abc import Iterable
from spacy.training import biluo_tags_to_spans
from itertools import combinations
import iobes

from zipfile import ZipFile
from pathlib import Path
from seqeval import scheme
from tqdm import autonotebook as tqdm

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

import sklearn_crfsuite

nlp = spacy.load("en_core_web_sm")


In [37]:
nlp = spacy.load("en_core_web_sm")
data_dir = Path("./data/teaching-dataset")
with (data_dir / "relation_extraction_text_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]
with (data_dir / "relation_extraction_references_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("references.txt") as f:
        labels = []
        for line in f.read().decode("utf-8").split("\n"):
            relations = []
            for relation in re.finditer(r"\(\((\d+),(\d+)\),\((\d+),(\d+)\)\)", line):
                relation = (
                    (int(relation.group(1)), int(relation.group(2))),
                    (int(relation.group(3)), int(relation.group(4))),
                )
                relations.append(relation)
            labels.append(relations)
assert len(sentences) == len(labels)
doc = nlp(Doc(nlp.vocab, words=sentences[0]))
doc

The longest serving spacecraft goes into retirement . 

In [22]:
train_sentences, val_sentences, train_labels, val_labels =  train_test_split(sentences, labels, test_size=0.2, random_state=42)
print(len(train_sentences))
print(len(val_sentences))

748
188


In [23]:
class is_causal_predictor:
    def __init__(self, sentences, labels, n=None):
        self.sentences = sentences
        self.labels = labels
        self.n = n
        
        if n != None:
            self.init_words(sentences, labels)
            self.causal_cues = self.get_causal_cues(self.n)
        else:
            self.init_causal_cues_best_n(sentences, labels)


    def init_words(self, sentences, labels):
        self.words = []
        self.nonCausalWords = []

        for label, sentence in zip(labels, sentences):
            if type(sentence) == list:
                sentence = ' '.join(sentence)
            if label != []: #if sentence is causal
                doc = nlp(sentence)
                wordsHelp = [token.text for token in doc if not token.is_stop and not token.is_punct and token.pos_ != "NOUN" and token.pos_ != "ADJ"] 
                self.words.extend(wordsHelp) #append all words to a list if they are NOT nouns & NOT punctuation & NOT adjectives

            else: #if sentence is NOT causal
                doc = nlp(sentence)
                wordsHelp = [token.text for token in doc if not token.is_stop and not token.is_punct and token.pos_ != "NOUN" and token.pos_ != "ADJ"] 
                self.nonCausalWords.extend(wordsHelp)  
    

    # returns n best causal cues
    def get_causal_cues(self, n):
        def flatten(lis): #pretty ugly solution but we have to flatten the list since every new sentence adds "[]" which Counter can't deal with
            for item in lis:
                if isinstance(item, Iterable) and not isinstance(item, str):
                    for x in flatten(item):
                        yield x
                else:        
                    yield item
        
        def get_n_lemmata(causal_freq, n):
            # sort words
            sorted_words = np.array(causal_freq.most_common(len(causal_freq)))[:,0]
            converted_return = []
            for word in sorted_words:
                # lemmatize
                doc = nlp(str(word))
                word = " ".join([token.lemma_ for token in doc])
                if not word in converted_return:
                    converted_return.append(word)
                # break if n lemmata found
                if len(converted_return)==n:
                    break
            return converted_return

        causal_freq = Counter(self.words)
        nonCausal_freq = Counter(self.nonCausalWords)
        for word in causal_freq:
            causal_freq[word] = causal_freq[word]/(nonCausal_freq[word]+1)
        return get_n_lemmata(causal_freq, n)


    def predict_causality(self, sentence):
        if type(sentence) == list: # convert to str
            sentence = ' '.join(sentence)
        matcher = Matcher(nlp.vocab)
        pattern = [[{"LEMMA": cue}] for cue in self.causal_cues]
        matcher.add("CAUSAL", pattern)
        doc = nlp(sentence)
        matches = matcher(doc)
        return bool(matches)


    # Find best value for n given a testset and initialize causal cues according to best n
    def init_causal_cues_best_n(self, sentences, labels, step_size=5):
        train_sentences, test_sentences, train_labels, test_labels =  train_test_split(sentences, labels, test_size=0.2, random_state=42)
        self.init_words(train_sentences, train_labels)

        new_labels = []
        for label in test_labels:
            if label != []:
                new_labels.append(1)
            else:
                new_labels.append(0)

        old_f1 = 0
        f1 = 0
        n = 0
        while f1 >= old_f1:
            n = n+step_size
            predictions = []

            # predict
            self.causal_cues = self.get_causal_cues(n)
            for sentence, l in zip(test_sentences, test_labels):
                p = self.predict_causality(sentence)
                predictions.append(p)

            # evaluate
            old_f1 = f1
            tp = sum([int(p) == 1 and int(l) == 1 for p, l in zip(predictions, new_labels)])
            fp = sum([int(p) == 1 and int(l) == 0 for p, l in zip(predictions, new_labels)])
            fn = sum([int(p) == 0 and int(l) == 1 for p, l in zip(predictions, new_labels)])
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1 = 2 * precision * recall / (precision + recall)
            x = 0

        self.f1 = old_f1 
        self.n = n - step_size # go back to best value for n
        self.causal_cues = self.get_causal_cues(self.n)


In [24]:
class SentenceGetterTrain(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, d, t) for w, p, d, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['DEP'].values.tolist(),
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [25]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    deptag = sent[i][2]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'deptag': deptag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        deptag1 = sent[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:deptag': deptag1,
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        # print(i)
        deptag1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:deptag': deptag1,
        })
    else:
        features['EOS'] = True
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, deptag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, deptag, label in sent]

def flatten(l):
    output = []
    for sublist in l:
        for item in sublist:
            output.append(item)

        output.append('')
    return output

In [26]:
# training ML-model with data from exercise 2
def train_step_2():
    data_dir = Path("./data/teaching-dataset")
    with (data_dir / "span_extraction_text_train.zip").open("rb") as file:
        zip_file = ZipFile(file)
        with zip_file.open("input.txt") as f:
            sentences = [
                sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
            ]
    with (data_dir / "span_extraction_references_train.zip").open("rb") as file:
        zip_file = ZipFile(file)
        with zip_file.open("references.txt") as f:
            labels = [
                sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
            ]

    sentence_number=[]
    words=[]
    tags=[]
    dependencies=[]
    events=[]
    for i in range(len(sentences)):
        doc = Doc(nlp.vocab, words=sentences[i])
        doc = nlp(doc)
        iterator = zip(doc, labels[i])
        for token, label in iterator:
            sentence_number.append(f'Sentence: {i}')
            words.append(str(token))
            tags.append(token.pos_)
            dependencies.append(token.dep_)
            events.append(label)

    train_data = {'Sentence #': sentence_number,'Word': words, 'POS': tags, 'DEP': dependencies, 'Tag': events}
    df = pd.DataFrame(data=train_data)
    # print(df)

    df.isnull().sum()

    df = df.fillna(method='ffill')
    df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique();

    df.groupby('Tag').size().reset_index(name='counts');

    X = df.drop('Tag', axis=1)
    v = DictVectorizer(sparse=False)
    X = v.fit_transform(X.to_dict('records'))
    y = df.Tag.values
    classes = np.unique(y)
    classes = classes.tolist()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
    X_train.shape, y_train.shape;

    getter = SentenceGetterTrain(df)
    sentences = getter.sentences

    X = [sent2features(s) for s in sentences]
    y = [sent2labels(s) for s in sentences]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=0)

    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train);

    y_pred = crf.predict(X_test)
    # print(metrics.flat_classification_report(y_pred=y_pred, y_true=y_test, labels = new_classes))
    # sollte eigentlich funktionieren tuts aber nicht
    flat_y_true = flatten(y_test)
    flat_y_pred = flatten(y_pred)
    # print(classification_report(flat_y_true, flat_y_pred))
    return(crf)

In [27]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, d) for w, p, d in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['DEP'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        

        # print(self.grouped.head(20))
        self.sentences = [s for s in self.grouped]
        
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

class span_extractor:
    def __init__(self):
        pass

    def create_dataframe(self, test_sentences):
        sentence_number=[]
        words=[]
        tags=[]
        dependencies=[]
        events=[]
        for i in range(len(test_sentences)):
            doc = Doc(nlp.vocab, words=test_sentences[i])
            doc = nlp(doc)
            for token in doc:
                sentence_number.append(f'Sentence: {i}')
                words.append(str(token))
                tags.append(token.pos_)
                dependencies.append(token.dep_)

        train_data = {'Sentence #': sentence_number,'Word': words, 'POS': tags, 'DEP': dependencies}
        df = pd.DataFrame(data=train_data)
        return df

    def getSentences(self, df):
        getter = SentenceGetter(df)
        sentences = getter.sentences
        return sentences
        # for sentence in sentences:
        #     print(sentence)

    def flatten(l):
        output = []
        for sublist in l:
            for item in sublist:
                output.append(item)

            output.append('')
        return output
    
    def cleanup(self, prediction):
        counter = 0
        for e in prediction:
            if e == 'B-EVENT':
                counter += 1

        if counter < 2:
            prediction = False

        return prediction

    def predict(self, sentence, crf):
        df = self.create_dataframe(sentence)
        sentences = self.getSentences(df)
        
        X = [sent2features(s) for s in sentences]
        y_pred = crf.predict(X)
        flat_y_pred = flatten(y_pred)
        prediction = self.cleanup(flat_y_pred)
        return prediction

In [28]:
class relation_classificator:
    def __init__(self):
        pass

    # [HELPER] adds event spans to doc
    def parse_sentence(self, sentence, spans):
        words = []
        tags = []
        for word, tag in zip(sentence, spans):
            words.append(word)
            tags.append(tag)
        doc = Doc(nlp.vocab, words=words)
        doc = nlp(doc)
        tags = iobes.bio_to_bilou(tags)
        doc.ents = biluo_tags_to_spans(doc, tags)
        return doc

    # [HELPER] is the token in between 2 events?
    def token_in_between_events(self, token, outer1, outer2):
        return  token.i <= outer1.start and token.i >= outer2.end or \
                token.i <= outer2.start and token.i >= outer1.end

    # [HELPER] is the token inside an event?
    def inside_event(self, event, token):
        return event.start <= token.i and event.end >= token.i

    # [HELPER] are both events inside a token's subtree?
    def events_inside_subtree(self, verb, event1, event2):
        a = False
        b = False
        for sub in verb.subtree:
            if self.inside_event(event1, sub):
                a = True
        for sub in verb.subtree:
            if self.inside_event(event2, sub):
                b = True
        return a and b

    # [HELPER] does the sentence induce a backwards relation?
    def backwards(self, verb, doc):
        keyword = None
        backwards = False
        for child in verb.children:
            if child.dep_ in ["agent"]:
                backwards = True
                keyword = child
            if child.text in ["from"]:
                backwards = True
                keyword = child
        next_word = doc[verb.i+1]
        if next_word.dep_ in ["aux"]:
            backwards = True
            keyword = next_word
        return backwards, keyword

    # [HELPER] what's the next event after a position?
    def get_next_event(self, doc, position):
        lowest_distance = float('inf')
        for event in doc.ents:
            if abs(event.start - position) <= lowest_distance and position > event.end or position < event.start:
                next_event = event
                lowest_distance = abs(event.start - position)
                return next_event

    # [HELPER] make predictions for structures like "x is cause of/for y"
    def handle_cause_of(self, predictions, doc):
        for token in doc:
            if token.text == "cause" and doc[token.i+1].text in ["of", "for"]:
                for event1, event2 in combinations(doc.ents, 2):
                    if self.token_in_between_events(token, event1, event2):
                        predictions.append(((event1.start, event1.end), (event2.start, event2.end)))
                if len(predictions) == 0:
                    effect = self.get_next_event(doc, token.i)
                    cause = self.get_next_event(doc, effect.end)
                    predictions.append(((cause.start, cause.end), (effect.start, effect.end)))
        return predictions

    # [HELPER] make predictions for structures like "x because/due/common_with y"
    def handle_because(self, predictions, doc):
        for token in doc:
            if token.text == "because" or token.text == "due" or token.text == "common" and doc[token.i+1].text == "with":
                for event1, event2 in combinations(doc.ents, 2):
                    if self.token_in_between_events(token, event1, event2):
                        predictions.append(((event2.start, event2.end), (event1.start, event1.end)))
                if len(predictions) == 0:
                    effect = self.get_next_event(doc, token.i)
                    cause = self.get_next_event(doc, effect.end)
                    predictions.append(((cause.start, cause.end), (effect.start, effect.end)))
        return predictions


    # [PREDICTOR] method to make causal relation predictions from given sentence and span string
    def predict(self, sentence, spans):
        doc = self.parse_sentence(sentence, spans)
        events = doc.ents
        predictions = []

        predictions = self.handle_cause_of(predictions, doc)
        predictions = self.handle_because(predictions, doc)

        # find verbs
        verb_pattern = [[{'POS': 'VERB'}]]
        verb_matcher = Matcher(nlp.vocab)
        verb_matcher.add("verbs", verb_pattern)
        matches = verb_matcher(doc)
        verbs = [(doc[start:end]) for _, start, end in matches]
        verbs = [doc[verb.start] for verb in verbs] # get tokens instead of spans

        # remove verbs inside events
        for verb in verbs:
            for event in events:
                if verb.i >= event.start and verb.i < event.end:
                    verbs.remove(verb)

        for event1, event2 in combinations(doc.ents, 2): # events are actually in order
            for verb in verbs:
                is_backwards, keyword = self.backwards(verb, doc)
                # predict if both events inside the verb's subtree and the verb is in between events
                if is_backwards:
                    if self.events_inside_subtree(verb, event1, event2) and self.token_in_between_events(keyword, event1, event2):
                        predictions.append(((event2.start, event2.end), (event1.start, event1.end)))
                else:
                    if self.events_inside_subtree(verb, event1, event2) and self.token_in_between_events(verb, event1, event2):
                        predictions.append(((event1.start, event1.end), (event2.start, event2.end)))
        
        # if there is no prediction yet, use a less strict rule
        if len(predictions) == 0:
            for event1, event2 in combinations(doc.ents, 2):
                for verb in verbs:
                    is_backwards, keyword = self.backwards(verb, doc) 
                    # predict if the verb is in between events - less strict
                    if is_backwards: 
                        if self.token_in_between_events(keyword, event1, event2):
                            predictions.append(((event2.start, event2.end), (event1.start, event1.end)))
                    else:
                        if self.token_in_between_events(verb, event1, event2):
                            predictions.append(((event1.start, event1.end), (event2.start, event2.end)))

        # default predictions if nothing worked: "everything causes everything"
        if len(predictions) == 0:
            for event1, event2 in combinations(doc.ents, 2):
                predictions.append(((event1.start, event1.end), (event2.start, event2.end)))

        predictions = set(predictions)

        return predictions

In [39]:
class pipeline:
    def __init__(self, train_sentences, train_labels):
        self.train_sentences = train_sentences
        self.train_labels = train_labels
        
        self.is_causal_predictor = is_causal_predictor(train_sentences, train_labels)
        self.span_extractor = span_extractor()
        self.relation_classificator = relation_classificator()

        self.crf = train_step_2()

    def predict(self, sentence):
        is_causal = self.is_causal_predictor.predict_causality(sentence)
        if is_causal:
            spans = self.span_extractor.predict([sentence], self.crf)
            if spans:
                prediction = self.relation_classificator.predict(sentence, spans)
            else:
                prediction = []
        else:
            prediction = []
        return prediction

In [40]:
pipe = pipeline(train_sentences, train_labels)

In [41]:
# pipe.predict(sentences[11])
val_predictions = []
for s in val_sentences:
    val_predictions.append(pipe.predict(s))

In [42]:
print(val_predictions)

[[], [], [], [], {((2, 3), (7, 10))}, [], [], [], [], [], [], [], [], [], {((1, 2), (24, 29))}, [], [], {((0, 2), (5, 10))}, {((0, 4), (11, 13)), ((0, 4), (26, 27)), ((0, 4), (20, 21)), ((0, 4), (48, 49)), ((0, 4), (32, 34)), ((0, 4), (39, 40))}, [], [], {((4, 6), (11, 19))}, {((21, 33), (13, 17)), ((19, 20), (13, 17))}, [], [], [], {((5, 7), (22, 27)), ((8, 16), (22, 27))}, {((19, 21), (27, 32))}, [], [], {((27, 29), (31, 35)), ((1, 7), (31, 35)), ((1, 7), (27, 29))}, [], [], {((12, 14), (28, 30)), ((12, 14), (15, 16)), ((15, 16), (28, 30))}, [], {((4, 10), (39, 47)), ((4, 10), (16, 17)), ((4, 10), (60, 63)), ((4, 10), (23, 26)), ((4, 10), (48, 52)), ((4, 10), (12, 14)), ((4, 10), (64, 67)), ((4, 10), (18, 19))}, {((7, 11), (35, 36)), ((7, 11), (38, 40)), ((7, 11), (15, 21)), ((7, 11), (22, 23)), ((7, 11), (33, 34)), ((7, 11), (24, 26)), ((7, 11), (27, 30)), ((7, 11), (12, 14))}, [], {((27, 30), (14, 16))}, [], {((0, 2), (14, 17)), ((0, 2), (4, 7))}, [], {((31, 33), (39, 41)), ((34, 3

In [43]:
def overlap(ref_event, pred_event):
    return max(ref_event[0], pred_event[0]) <= min(ref_event[1], pred_event[1])


def evaluate_pair(reference, prediction):
    ref_cause, ref_effect = reference
    pred_cause, pred_effect = prediction
    if ref_cause == pred_cause and ref_effect == pred_effect:
        return 1
    elif overlap(ref_cause, pred_cause) and overlap(ref_effect, pred_effect):
        return 0.5
    return 0

def precision(tp, fp):
    if not tp:
        return 0
    return tp / (tp + fp)

def recall(tp, fn):
    if not tp:
        return 0
    return tp / (tp + fn)

def f1(tp, fp, fn):
    if not tp:
        return 0
    return 2 * tp / (2 * tp + fp + fn)

def evaluate(references, predictions):
    tps, fps, fns = [], [], []
    for reference, prediction in zip(references, predictions):
        tp, fp, fn = 0, 0, 0
        remaining_references = set(reference)
        for pred in prediction:
            for ref in remaining_references:
                score = evaluate_pair(ref, pred)
                if score:
                    tp += score
                    remaining_references.remove(ref)
                    break
            else:
                fp += 1
        fn += len(remaining_references)
        tps.append(tp)
        fps.append(fp)
        fns.append(fn)

    macro_prec = sum([precision(tp, fp) for tp, fp in zip(tps, fps)]) / len(tps)
    macro_rec = sum([recall(tp, fn) for tp, fn in zip(tps, fns)]) / len(tps)
    macro_f1 = sum([f1(tp, fp, fn) for tp, fp, fn in zip(tps, fps, fns)]) / len(tps)
    micro_prec = precision(sum(tps), sum(fps))
    micro_rec = recall(sum(tps), sum(fns))
    micro_f1 = f1(sum(tps), sum(fps), sum(fns))
    return {
        "macro": {"precision": macro_prec, "recall": macro_rec, "f1": macro_f1},
        "micro": {"precision": micro_prec, "recall": micro_rec, "f1": micro_f1},
    }



In [44]:
evaluate(val_labels, val_predictions)

{'macro': {'precision': 0.2774822695035461,
  'recall': 0.29476950354609927,
  'f1': 0.2816193853427896},
 'micro': {'precision': 0.4880952380952381,
  'recall': 0.4659090909090909,
  'f1': 0.47674418604651164}}

In [45]:
data_dir = Path("./data/teaching-dataset")
with (data_dir / "relation_extraction_text_test.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        test_sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]

pipe = pipeline(sentences, labels)
test_predictions = []
for s in test_sentences:
    test_predictions.append(pipe.predict(s))

with open("predictions.txt", "w") as f:
    f.write("\n".join(",".join(str(relation) for relation in prediction) for prediction in test_predictions).replace(" ", ""))#
    
# this causes problems if last line is empty!

In [17]:
print(len(test_sentences))
print(len(test_predictions))

196
196
